In [1]:
#conda install -c anaconda pandas-datareader

In [2]:
#conda install -c rodgomesc pip-ccxt

In [3]:
# importing the libraries
import pandas as pd
import numpy as np
import pandas_datareader as pdr
from functools import reduce  

# Importing and cleaning the data

## Government's Monetary Data

In [4]:
#onthly average amount outstanding of total sterling notes and coin in circulation outside the BoE
data1 = pd.read_csv(r'C:\Users\greea\OneDrive\Documenti\Research Projects\Crypto Macro Research Project\BoE_LPMAVAB_and_LPMB8H4.csv')
data1.rename(columns={'Monthly average amount outstanding of total sterling notes and coin in circulation, excluding backing assets for commercial banknote issue in Scotland and Northern Ireland (in sterling millions) seasonally adjusted              [a] [b] [c] [d] [e] [f]             LPMAVAB':'Monthly_Out',
                      'Monthly average amount outstanding of total sterling notes and coin in circulation, excluding backing assets for commercial banknote issue in Scotland and Northern Ireland adjusted for specific non-seasonal events total (in sterling millions) seasonally adjusted              [b] [g] [d] [e]             LPMB8H4': 'Monthly_Out_spec'}
             ,inplace = True)
data1.info()
data1.columns
data1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Date              184 non-null    object
 1   Monthly_Out       184 non-null    int64 
 2   Monthly_Out_spec  184 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 4.4+ KB


,Date,Monthly_Out,Monthly_Out_spec
0,30 Apr 23,94083,93980
1,31 Mar 23,94295,94133
2,28 Feb 23,94452,94308
3,31 Jan 23,94461,94189
4,31 Dec 22,94042,94095


In [5]:
#Monthly changes in monthly average of amounts outstanding (on Wednesdays) of Bank of England Banking Department sterling reserves balance liabilities 
data2 = pd.read_csv(r'C:\Users\greea\OneDrive\Documenti\Research Projects\Crypto Macro Research Project\BoE_LPMB23.csv')
data2.rename(columns={'Monthly changes in monthly average of amounts outstanding (on Wednesdays) of Bank of England Banking Department sterling reserves balance liabilities (in sterling millions) not seasonally adjusted                          LPMBL23':'Monthly_Out_BoE'},inplace = True)
data2.info()
data2.columns
data2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date             184 non-null    object
 1   Monthly_Out_BoE  184 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.0+ KB


,Date,Monthly_Out_BoE
0,30 Apr 23,-13010
1,31 Mar 23,-6188
2,28 Feb 23,-7171
3,31 Jan 23,-14368
4,31 Dec 22,-16029


In [6]:
#Merging the two datasets
data =  pd.merge(data1, data2, on=["Date"])
data['Date'] = pd.to_datetime(data['Date'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184 entries, 0 to 183
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              184 non-null    datetime64[ns]
 1   Monthly_Out       184 non-null    int64         
 2   Monthly_Out_spec  184 non-null    int64         
 3   Monthly_Out_BoE   184 non-null    int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 7.2 KB


In [7]:
data.head()

,Date,Monthly_Out,Monthly_Out_spec,Monthly_Out_BoE
0,2023-04-30,94083,93980,-13010
1,2023-03-31,94295,94133,-6188
2,2023-02-28,94452,94308,-7171
3,2023-01-31,94461,94189,-14368
4,2022-12-31,94042,94095,-16029


In [8]:
data.tail()

,Date,Monthly_Out,Monthly_Out_spec,Monthly_Out_BoE
179,2008-05-31,49826,49853,864
180,2008-04-30,49666,49676,958
181,2008-03-31,49550,49562,2888
182,2008-02-29,49223,49229,-978
183,2008-01-31,48888,48895,-90


In [9]:
#Importing the Federal Reserve Bank of St. Louis M2 money supply
fred_m2 = pdr.get_data_fred('M2', start='2008-01-31', end='2023-04-30')
fred_m2.reset_index(inplace=True)
fred_m2.rename(columns={'DATE':'Date'},inplace = True)
fred_m2['Date'] = pd.to_datetime(fred_m2['Date'])
fred_m2.head()

,Date,M2
0,2008-02-04,7567.8
1,2008-02-11,7576.9
2,2008-02-18,7579.6
3,2008-02-25,7611.0
4,2008-03-03,7620.8


In [10]:
fred_m2.tail()

,Date,M2
674,2021-01-04,19040.0
675,2021-01-11,19411.1
676,2021-01-18,19425.1
677,2021-01-25,19376.4
678,2021-02-01,19370.8


In [11]:
fred_m2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679 entries, 0 to 678
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    679 non-null    datetime64[ns]
 1   M2      679 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 10.7 KB


In [12]:
fred_m2.set_index('Date', inplace=True)
# Resample weekly data to monthly using the mean value
fred_m2_monthly = fred_m2.resample('M').mean()

# Reset the index to make the 'Date' column a regular column again
fred_m2_monthly.reset_index(inplace=True)
fred_m2_monthly.head()

,Date,M2
0,2008-02-29,7583.825
1,2008-03-31,7652.160
2,2008-04-30,7700.050
3,2008-05-31,7706.975
4,2008-06-30,7728.500


In [13]:
fred_m2_monthly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    157 non-null    datetime64[ns]
 1   M2      157 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.6 KB


In [14]:
#importing M2 from local machine since above data stops at 2021
fred_m2_23 =  pd.read_csv(r'C:\Users\greea\OneDrive\Documenti\Research Projects\Crypto Macro Research Project\M2SL.csv')
fred_m2_23.rename(columns={'DATE':'Date', 'M2SL': 'M2_FRED'},inplace = True)
fred_m2_23['Date'] = pd.to_datetime(fred_m2_23['Date'])
fred_m2_23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     183 non-null    datetime64[ns]
 1   M2_FRED  183 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.0 KB


In [15]:
fred_m2_23.head()

,Date,M2_FRED
0,2008-02-01,7590.6
1,2008-03-01,7656.2
2,2008-04-01,7699.2
3,2008-05-01,7711.2
4,2008-06-01,7728.9


In [16]:
"""Since the data from the FRED is from the first day of the month but most sources capture from the last day of the month, resample will be used to shift the date so that a merge operation can be performed"""

fred_m2_23.set_index('Date', inplace=True)
# Resample weekly data to monthly using the mean value
fred_m2_23_monthly = fred_m2_23.resample('M').mean()
# Reset the index to make the 'Date' column a regular column again
fred_m2_23_monthly.reset_index(inplace=True)
fred_m2_23_monthly.head()


,Date,M2_FRED
0,2008-02-29,7590.6
1,2008-03-31,7656.2
2,2008-04-30,7699.2
3,2008-05-31,7711.2
4,2008-06-30,7728.9


In [17]:
fred_m2_23_monthly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     183 non-null    datetime64[ns]
 1   M2_FRED  183 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.0 KB


In [18]:
#Merging the two data to fred_m2_monthly
tot_data_v1 =  pd.merge(data, fred_m2_monthly, on=["Date"])
tot_data_v1.head()

,Date,Monthly_Out,Monthly_Out_spec,Monthly_Out_BoE,M2
0,2021-02-28,93171,93138,20519,19370.80
1,2021-01-31,92260,92552,1847,19313.15
2,2020-12-31,90265,90721,3092,19057.00
3,2020-11-30,89837,90124,19877,18920.64
4,2020-10-31,89399,89534,21056,18696.60


In [19]:
tot_data_v1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157 entries, 0 to 156
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              157 non-null    datetime64[ns]
 1   Monthly_Out       157 non-null    int64         
 2   Monthly_Out_spec  157 non-null    int64         
 3   Monthly_Out_BoE   157 non-null    int64         
 4   M2                157 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 7.4 KB


In [20]:
# Merging BoE data with FRED data up to 2023
tot_data_v2 =  pd.merge(data, fred_m2_23_monthly, on=["Date"])
tot_data_v2.head()

,Date,Monthly_Out,Monthly_Out_spec,Monthly_Out_BoE,M2_FRED
0,2023-04-30,94083,93980,-13010,20673.1
1,2023-03-31,94295,94133,-6188,20840.3
2,2023-02-28,94452,94308,-7171,21076.9
3,2023-01-31,94461,94189,-14368,21212.6
4,2022-12-31,94042,94095,-16029,21358.1


In [21]:
tot_data_v2.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183 entries, 0 to 182
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              183 non-null    datetime64[ns]
 1   Monthly_Out       183 non-null    int64         
 2   Monthly_Out_spec  183 non-null    int64         
 3   Monthly_Out_BoE   183 non-null    int64         
 4   M2_FRED           183 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 8.6 KB


## Crypto Data


### Layer 1

In [22]:
#importing bitcoin data
btc =  pd.read_csv(r'C:\Users\greea\OneDrive\Documenti\Research Projects\Crypto Macro Research Project\BTC-USD.csv')
btc['Date'] = pd.to_datetime(btc['Date'])
btc_price = btc.loc[:,['Date','Close']]
btc_price.rename(columns={'Close':'BTC_Close'}, inplace = True)
btc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       103 non-null    datetime64[ns]
 1   Open       103 non-null    float64       
 2   High       103 non-null    float64       
 3   Low        103 non-null    float64       
 4   Close      103 non-null    float64       
 5   Adj Close  103 non-null    float64       
 6   Volume     103 non-null    int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 5.8 KB


In [23]:
btc_price.head()

,Date,BTC_Close
0,2014-10-01,338.321014
1,2014-11-01,378.046997
2,2014-12-01,320.192993
3,2015-01-01,217.464005
4,2015-02-01,254.263000


In [24]:
#importing ethereum data
eth =  pd.read_csv(r'C:\Users\greea\OneDrive\Documenti\Research Projects\Crypto Macro Research Project\ETH-USD.csv')
eth['Date'] = pd.to_datetime(eth['Date'])
eth_price = eth.loc[:,['Date','Close']]
eth_price.rename(columns={'Close':'ETH_Close'}, inplace = True)
eth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       65 non-null     datetime64[ns]
 1   Open       65 non-null     float64       
 2   High       65 non-null     float64       
 3   Low        65 non-null     float64       
 4   Close      65 non-null     float64       
 5   Adj Close  65 non-null     float64       
 6   Volume     65 non-null     int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 3.7 KB


In [25]:
eth_price.head()

,Date,ETH_Close
0,2017-12-01,756.732971
1,2018-01-01,1118.310059
2,2018-02-01,855.198975
3,2018-03-01,396.457001
4,2018-04-01,669.924011


In [26]:
#importing binance coin data
bnb =  pd.read_csv(r'C:\Users\greea\OneDrive\Documenti\Research Projects\Crypto Macro Research Project\BNB-USD.csv')
bnb['Date'] = pd.to_datetime(bnb['Date'])
bnb_price = bnb.loc[:,['Date','Close']]
bnb_price.rename(columns={'Close':'BNB_Close'}, inplace = True)
bnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       65 non-null     datetime64[ns]
 1   Open       65 non-null     float64       
 2   High       65 non-null     float64       
 3   Low        65 non-null     float64       
 4   Close      65 non-null     float64       
 5   Adj Close  65 non-null     float64       
 6   Volume     65 non-null     int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 3.7 KB


In [27]:
bnb_price.head()

,Date,BNB_Close
0,2017-12-01,8.63558
1,2018-01-01,11.14500
2,2018-02-01,10.43820
3,2018-03-01,11.05590
4,2018-04-01,14.31210


In [28]:
"""Since the data from the Yahoo data is from the first day of the month but most sources capture from the last day of the month, resample will be used to shift the date so that a merge operation can be performed"""

btc_price.set_index('Date', inplace=True)
# Resample weekly data to monthly using the mean value
btc_price_monthly = btc_price.resample('M').mean()
# Reset the index to make the 'Date' column a regular column again
btc_price_monthly.reset_index(inplace=True)

eth_price.set_index('Date', inplace=True)
# Resample weekly data to monthly using the mean value
eth_price_monthly = eth_price.resample('M').mean()
# Reset the index to make the 'Date' column a regular column again
eth_price_monthly.reset_index(inplace=True)

bnb_price.set_index('Date', inplace=True)
# Resample weekly data to monthly using the mean value
bnb_price_monthly = bnb_price.resample('M').mean()
# Reset the index to make the 'Date' column a regular column again
bnb_price_monthly.reset_index(inplace=True)

In [29]:
# merging treasuries and crypto basket data
tot_data_L1 =  tot_data_v2.merge(btc_price_monthly.merge(eth_price_monthly.merge(bnb_price_monthly, on='Date'), on='Date'), on='Date')
tot_data_L1.rename(columns={'Close':'BTC_Close','Close_x':'ETH_Close','Close_y':'BNB_Close'},inplace = True)
tot_data_L1.head()

,Date,Monthly_Out,Monthly_Out_spec,Monthly_Out_BoE,M2_FRED,BTC_Close,ETH_Close,BNB_Close
0,2023-04-30,94083,93980,-13010,20673.1,29268.806641,1876.924316,337.645538
1,2023-03-31,94295,94133,-6188,20840.3,28478.484375,1822.022095,317.040009
2,2023-02-28,94452,94308,-7171,21076.9,23147.353516,1605.895142,301.474823
3,2023-01-31,94461,94189,-14368,21212.6,23139.283203,1586.535400,312.361053
4,2022-12-31,94042,94095,-16029,21358.1,16547.496094,1196.771240,246.349197


In [30]:
tot_data_L1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65 entries, 0 to 64
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              65 non-null     datetime64[ns]
 1   Monthly_Out       65 non-null     int64         
 2   Monthly_Out_spec  65 non-null     int64         
 3   Monthly_Out_BoE   65 non-null     int64         
 4   M2_FRED           65 non-null     float64       
 5   BTC_Close         65 non-null     float64       
 6   ETH_Close         65 non-null     float64       
 7   BNB_Close         65 non-null     float64       
dtypes: datetime64[ns](1), float64(4), int64(3)
memory usage: 4.6 KB


### Layer 2

In [31]:
#importing matic data
matic =  pd.read_csv(r'C:\Users\greea\OneDrive\Documenti\Research Projects\Crypto Macro Research Project\MATIC-USD.csv')
matic['Date'] = pd.to_datetime(matic['Date'])
matic_price = matic.loc[:,['Date','Close']]
matic_price.rename(columns={'Close':'MATIC_Close'}, inplace = True)
matic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       48 non-null     datetime64[ns]
 1   Open       48 non-null     float64       
 2   High       48 non-null     float64       
 3   Low        48 non-null     float64       
 4   Close      48 non-null     float64       
 5   Adj Close  48 non-null     float64       
 6   Volume     48 non-null     int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 2.8 KB


In [32]:
matic_price.head()

,Date,MATIC_Close
0,2019-05-01,0.022288
1,2019-06-01,0.021971
2,2019-07-01,0.011374
3,2019-08-01,0.011846
4,2019-09-01,0.010847


In [33]:
#importing imx data
imx =  pd.read_csv(r'C:\Users\greea\OneDrive\Documenti\Research Projects\Crypto Macro Research Project\IMX10603-USD.csv')
imx['Date'] = pd.to_datetime(imx['Date'])
imx_price = imx.loc[:,['Date','Close']]
imx_price.rename(columns={'Close':'IMX_Close'}, inplace = True)
imx.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       17 non-null     datetime64[ns]
 1   Open       17 non-null     float64       
 2   High       17 non-null     float64       
 3   Low        17 non-null     float64       
 4   Close      17 non-null     float64       
 5   Adj Close  17 non-null     float64       
 6   Volume     17 non-null     int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 1.1 KB


In [34]:
imx_price.head()

,Date,IMX_Close
0,2021-12-01,4.895250
1,2022-01-01,2.701512
2,2022-02-01,1.557913
3,2022-03-01,2.345393
4,2022-04-01,1.356964


In [35]:
#importing optimism data
optimism =  pd.read_csv(r'C:\Users\greea\OneDrive\Documenti\Research Projects\Crypto Macro Research Project\OP-USD.csv')
optimism['Date'] = pd.to_datetime(optimism['Date'])
optimism_price = optimism.loc[:,['Date','Close']]
optimism_price.rename(columns={'Close':'OP_Close'}, inplace = True)
optimism.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       13 non-null     datetime64[ns]
 1   Open       13 non-null     float64       
 2   High       13 non-null     float64       
 3   Low        13 non-null     float64       
 4   Close      13 non-null     float64       
 5   Adj Close  13 non-null     float64       
 6   Volume     13 non-null     int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 856.0 bytes


In [36]:
optimism_price.head()

,Date,OP_Close
0,2022-04-01,0.000587
1,2022-05-01,0.000489
2,2022-06-01,0.000333
3,2022-07-01,0.000430
4,2022-08-01,0.000418


In [37]:
"""Since the data from the Yahoo data is from the first day of the month but most sources capture from the last day of the month, resample will be used to shift the date so that a merge operation can be performed"""
matic_price.set_index('Date', inplace=True)
# Resample weekly data to monthly using the mean value
matic_price_monthly = matic_price.resample('M').mean()
# Reset the index to make the 'Date' column a regular column again
matic_price_monthly.reset_index(inplace=True) 

imx_price.set_index('Date', inplace=True)
# Resample weekly data to monthly using the mean value
imx_price_monthly = imx_price.resample('M').mean()
# Reset the index to make the 'Date' column a regular column again
imx_price_monthly.reset_index(inplace=True)

optimism_price.set_index('Date', inplace=True)
# Resample weekly data to monthly using the mean value
optimism_price_monthly = optimism_price.resample('M').mean()
# Reset the index to make the 'Date' column a regular column again
optimism_price_monthly.reset_index(inplace=True)

In [38]:
# merging treasuries and crypto basket data
tot_data_L2 =  tot_data_v2.merge(imx_price_monthly.merge(matic_price_monthly.merge(optimism_price_monthly, on='Date'), on='Date'), on='Date')
tot_data_L2.rename(columns={'Close':'MATIC_Close','Close_x':'IMX_Close','Close_y':'OP_Close'},inplace = True)
tot_data_L2.head()

,Date,Monthly_Out,Monthly_Out_spec,Monthly_Out_BoE,M2_FRED,IMX_Close,MATIC_Close,OP_Close
0,2023-04-30,94083,93980,-13010,20673.1,0.990445,0.984081,2.175580
1,2023-03-31,94295,94133,-6188,20840.3,1.136617,1.119162,2.286609
2,2023-02-28,94452,94308,-7171,21076.9,1.060570,1.195239,2.700007
3,2023-01-31,94461,94189,-14368,21212.6,0.750705,1.111333,2.169502
4,2022-12-31,94042,94095,-16029,21358.1,0.383502,0.758517,0.917185


In [39]:
tot_data_L2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 0 to 12
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              13 non-null     datetime64[ns]
 1   Monthly_Out       13 non-null     int64         
 2   Monthly_Out_spec  13 non-null     int64         
 3   Monthly_Out_BoE   13 non-null     int64         
 4   M2_FRED           13 non-null     float64       
 5   IMX_Close         13 non-null     float64       
 6   MATIC_Close       13 non-null     float64       
 7   OP_Close          13 non-null     float64       
dtypes: datetime64[ns](1), float64(4), int64(3)
memory usage: 936.0 bytes


### Defi

In [40]:
#importing uniswap data
uniswap =  pd.read_csv(r'C:\Users\greea\OneDrive\Documenti\Research Projects\Crypto Macro Research Project\UNI7083-USD.csv')
uniswap['Date'] = pd.to_datetime(uniswap['Date'])
uniswap_price = uniswap.loc[:,['Date','Close']]
uniswap_price.rename(columns={'Close':'UNI_Close'}, inplace = True)
uniswap.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       31 non-null     datetime64[ns]
 1   Open       31 non-null     float64       
 2   High       31 non-null     float64       
 3   Low        31 non-null     float64       
 4   Close      31 non-null     float64       
 5   Adj Close  31 non-null     float64       
 6   Volume     31 non-null     int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 1.8 KB


In [41]:
uniswap_price.head()

,Date,UNI_Close
0,2020-10-01,2.266428
1,2020-11-01,3.769074
2,2020-12-01,5.168337
3,2021-01-01,17.666294
4,2021-02-01,22.189560


In [42]:
#importing curve data
curve =  pd.read_csv(r'C:\Users\greea\OneDrive\Documenti\Research Projects\Crypto Macro Research Project\CRV-USD.csv')
curve['Date'] = pd.to_datetime(curve['Date'])
curve_price = curve.loc[:,['Date','Close']]
curve_price.rename(columns={'Close':'CRV_Close'}, inplace = True)
curve.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       32 non-null     datetime64[ns]
 1   Open       32 non-null     float64       
 2   High       32 non-null     float64       
 3   Low        32 non-null     float64       
 4   Close      32 non-null     float64       
 5   Adj Close  32 non-null     float64       
 6   Volume     32 non-null     int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 1.9 KB


In [43]:
curve_price.head()

,Date,CRV_Close
0,2020-09-01,0.947779
1,2020-10-01,0.416495
2,2020-11-01,0.675537
3,2020-12-01,0.628331
4,2021-01-01,2.524605


In [44]:
#importing aave data
aave =  pd.read_csv(r'C:\Users\greea\OneDrive\Documenti\Research Projects\Crypto Macro Research Project\AAVE-USD.csv')
aave['Date'] = pd.to_datetime(aave['Date'])
aave_price = aave.loc[:,['Date','Close']]
aave_price.rename(columns={'Close':'AAVE_Close'}, inplace = True)
aave.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       30 non-null     datetime64[ns]
 1   Open       30 non-null     float64       
 2   High       30 non-null     float64       
 3   Low        30 non-null     float64       
 4   Close      30 non-null     float64       
 5   Adj Close  30 non-null     float64       
 6   Volume     30 non-null     int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 1.8 KB


In [45]:
aave_price.head()

,Date,AAVE_Close
0,2020-11-01,76.419533
1,2020-12-01,87.527481
2,2021-01-01,300.062561
3,2021-02-01,344.869995
4,2021-03-01,377.008148


In [46]:
"""Since the data from the Yahoo data is from the first day of the month but most sources capture from the last day of the month, resample will be used to shift the date so that a merge operation can be performed"""
uniswap_price.set_index('Date', inplace=True)
# Resample weekly data to monthly using the mean value
uniswap_price_monthly = uniswap_price.resample('M').mean()
# Reset the index to make the 'Date' column a regular column again
uniswap_price_monthly.reset_index(inplace=True)

curve_price.set_index('Date', inplace=True)
# Resample weekly data to monthly using the mean value
curve_price_monthly = curve_price.resample('M').mean()
# Reset the index to make the 'Date' column a regular column again
curve_price_monthly.reset_index(inplace=True)

aave_price.set_index('Date', inplace=True)
# Resample weekly data to monthly using the mean value
aave_price_monthly = aave_price.resample('M').mean()
# Reset the index to make the 'Date' column a regular column again
aave_price_monthly.reset_index(inplace=True)

In [47]:
# merging treasuries and crypto basket data
tot_data_defi =  tot_data_v2.merge(uniswap_price_monthly.merge(curve_price_monthly.merge(aave_price_monthly, on='Date'), on='Date'), on='Date')
tot_data_defi.rename(columns={'Close':'UNI_Close','Close_x':'CRV_Close','Close_y':'AAVE_Close'},inplace = True)
tot_data_defi.head()

,Date,Monthly_Out,Monthly_Out_spec,Monthly_Out_BoE,M2_FRED,UNI_Close,CRV_Close,AAVE_Close
0,2023-04-30,94083,93980,-13010,20673.1,5.468202,0.910968,70.587372
1,2023-03-31,94295,94133,-6188,20840.3,6.061128,0.935072,74.199478
2,2023-02-28,94452,94308,-7171,21076.9,6.489455,0.992869,77.554825
3,2023-01-31,94461,94189,-14368,21212.6,6.568031,1.018499,83.018906
4,2022-12-31,94042,94095,-16029,21358.1,5.162892,0.524823,51.979134


In [48]:
tot_data_defi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              30 non-null     datetime64[ns]
 1   Monthly_Out       30 non-null     int64         
 2   Monthly_Out_spec  30 non-null     int64         
 3   Monthly_Out_BoE   30 non-null     int64         
 4   M2_FRED           30 non-null     float64       
 5   UNI_Close         30 non-null     float64       
 6   CRV_Close         30 non-null     float64       
 7   AAVE_Close        30 non-null     float64       
dtypes: datetime64[ns](1), float64(4), int64(3)
memory usage: 2.1 KB


### Gaming

In [49]:
#importing render data
render =  pd.read_csv(r'C:\Users\greea\OneDrive\Documenti\Research Projects\Crypto Macro Research Project\RNDR-USD.csv')
render['Date'] = pd.to_datetime(render['Date'])
render_price = render.loc[:,['Date','Close']]
render_price.rename(columns={'Close':'RNDR_Close'}, inplace = True)
render.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       34 non-null     datetime64[ns]
 1   Open       34 non-null     float64       
 2   High       34 non-null     float64       
 3   Low        34 non-null     float64       
 4   Close      34 non-null     float64       
 5   Adj Close  34 non-null     float64       
 6   Volume     34 non-null     int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 2.0 KB


In [50]:
render_price.head()

,Date,RNDR_Close
0,2020-07-01,0.084505
1,2020-08-01,0.223215
2,2020-09-01,0.133926
3,2020-10-01,0.115679
4,2020-11-01,0.121257


In [51]:
#importing decentraland data
decentraland =  pd.read_csv(r'C:\Users\greea\OneDrive\Documenti\Research Projects\Crypto Macro Research Project\MANA-USD.csv')
decentraland['Date'] = pd.to_datetime(decentraland['Date'])
decentraland_price = decentraland.loc[:,['Date','Close']]
decentraland_price.rename(columns={'Close':'MANA_Close'}, inplace = True)
decentraland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       65 non-null     datetime64[ns]
 1   Open       65 non-null     float64       
 2   High       65 non-null     float64       
 3   Low        65 non-null     float64       
 4   Close      65 non-null     float64       
 5   Adj Close  65 non-null     float64       
 6   Volume     65 non-null     int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 3.7 KB


In [52]:
decentraland_price.head()

,Date,MANA_Close
0,2017-12-01,0.101999
1,2018-01-01,0.127432
2,2018-02-01,0.105240
3,2018-03-01,0.070817
4,2018-04-01,0.154917


In [53]:
#importing axie data
axie =  pd.read_csv(r'C:\Users\greea\OneDrive\Documenti\Research Projects\Crypto Macro Research Project\AXS-USD.csv')
axie['Date'] = pd.to_datetime(axie['Date'])
axie_price = axie.loc[:,['Date','Close']]
axie_price.rename(columns={'Close':'AXS_Close'}, inplace = True)
axie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       29 non-null     datetime64[ns]
 1   Open       29 non-null     float64       
 2   High       29 non-null     float64       
 3   Low        29 non-null     float64       
 4   Close      29 non-null     float64       
 5   Adj Close  29 non-null     float64       
 6   Volume     29 non-null     int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 1.7 KB


In [54]:
axie_price.head()

,Date,AXS_Close
0,2020-12-01,0.538320
1,2021-01-01,0.805305
2,2021-02-01,1.395820
3,2021-03-01,5.697310
4,2021-04-01,10.184638


In [55]:
"""Since the data from the Yahoo data is from the first day of the month but most sources capture from the last day of the month, resample will be used to shift the date so that a merge operation can be performed"""
render_price.set_index('Date', inplace=True)
# Resample weekly data to monthly using the mean value
render_price_monthly = render_price.resample('M').mean()
# Reset the index to make the 'Date' column a regular column again
render_price_monthly.reset_index(inplace=True)

decentraland_price.set_index('Date', inplace=True)
# Resample weekly data to monthly using the mean value
decentraland_price_monthly = decentraland_price.resample('M').mean()
# Reset the index to make the 'Date' column a regular column again
decentraland_price_monthly.reset_index(inplace=True)

axie_price.set_index('Date', inplace=True)
# Resample weekly data to monthly using the mean value
axie_price_monthly = axie_price.resample('M').mean()
# Reset the index to make the 'Date' column a regular column again
axie_price_monthly.reset_index(inplace=True)

In [56]:
# merging treasuries and crypto basket data
tot_data_gaming =  tot_data_v2.merge(render_price_monthly.merge(decentraland_price_monthly.merge(axie_price_monthly, on='Date'), on='Date'), on='Date')
tot_data_gaming.rename(columns={'Close':'RNDR_Close','Close_x':'MANA_Close','Close_y':'AXS_Close'},inplace = True)
tot_data_gaming.head()

,Date,Monthly_Out,Monthly_Out_spec,Monthly_Out_BoE,M2_FRED,RNDR_Close,MANA_Close,AXS_Close
0,2023-04-30,94083,93980,-13010,20673.1,2.394505,0.545592,7.936606
1,2023-03-31,94295,94133,-6188,20840.3,1.373936,0.588636,8.388087
2,2023-02-28,94452,94308,-7171,21076.9,1.455941,0.626488,9.607596
3,2023-01-31,94461,94189,-14368,21212.6,1.410599,0.750738,10.900343
4,2022-12-31,94042,94095,-16029,21358.1,0.402452,0.297013,6.041958


In [57]:
tot_data_gaming.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 0 to 28
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              29 non-null     datetime64[ns]
 1   Monthly_Out       29 non-null     int64         
 2   Monthly_Out_spec  29 non-null     int64         
 3   Monthly_Out_BoE   29 non-null     int64         
 4   M2_FRED           29 non-null     float64       
 5   RNDR_Close        29 non-null     float64       
 6   MANA_Close        29 non-null     float64       
 7   AXS_Close         29 non-null     float64       
dtypes: datetime64[ns](1), float64(4), int64(3)
memory usage: 2.0 KB


# Exporting the data for analysis

In [58]:
"""Data from different sources has different time range, here i will perform an outer merge to include all the rows from each timeseries"""

dfList = [tot_data_v2, 
          btc_price_monthly, 
          eth_price_monthly, 
          bnb_price_monthly, 
          matic_price_monthly, 
          imx_price_monthly, 
          optimism_price_monthly, 
          uniswap_price_monthly, 
          curve_price_monthly, 
          aave_price_monthly, 
          render_price_monthly, 
          decentraland_price_monthly,
          axie_price_monthly]
all_data = reduce(lambda left, right:     # Merge DataFrames in list
                     pd.merge(left , right,
                              on = ["Date"],
                              how = "outer"),
                     dfList)
all_data.head()   


,Date,Monthly_Out,Monthly_Out_spec,Monthly_Out_BoE,M2_FRED,BTC_Close,ETH_Close,BNB_Close,MATIC_Close,IMX_Close,OP_Close,UNI_Close,CRV_Close,AAVE_Close,RNDR_Close,MANA_Close,AXS_Close
0,2023-04-30,94083,93980,-13010,20673.1,29268.806641,1876.924316,337.645538,0.984081,0.990445,2.175580,5.468202,0.910968,70.587372,2.394505,0.545592,7.936606
1,2023-03-31,94295,94133,-6188,20840.3,28478.484375,1822.022095,317.040009,1.119162,1.136617,2.286609,6.061128,0.935072,74.199478,1.373936,0.588636,8.388087
2,2023-02-28,94452,94308,-7171,21076.9,23147.353516,1605.895142,301.474823,1.195239,1.060570,2.700007,6.489455,0.992869,77.554825,1.455941,0.626488,9.607596
3,2023-01-31,94461,94189,-14368,21212.6,23139.283203,1586.535400,312.361053,1.111333,0.750705,2.169502,6.568031,1.018499,83.018906,1.410599,0.750738,10.900343
4,2022-12-31,94042,94095,-16029,21358.1,16547.496094,1196.771240,246.349197,0.758517,0.383502,0.917185,5.162892,0.524823,51.979134,0.402452,0.297013,6.041958


In [59]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183 entries, 0 to 182
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              183 non-null    datetime64[ns]
 1   Monthly_Out       183 non-null    int64         
 2   Monthly_Out_spec  183 non-null    int64         
 3   Monthly_Out_BoE   183 non-null    int64         
 4   M2_FRED           183 non-null    float64       
 5   BTC_Close         103 non-null    float64       
 6   ETH_Close         65 non-null     float64       
 7   BNB_Close         65 non-null     float64       
 8   MATIC_Close       48 non-null     float64       
 9   IMX_Close         17 non-null     float64       
 10  OP_Close          13 non-null     float64       
 11  UNI_Close         31 non-null     float64       
 12  CRV_Close         32 non-null     float64       
 13  AAVE_Close        30 non-null     float64       
 14  RNDR_Close        34 non-n

In [60]:
with pd.ExcelWriter('Crypto_Macro_Research.xlsx') as writer:
    all_data.to_excel(writer, sheet_name='All_data')
    tot_data_v2.to_excel(writer, sheet_name='Treasuries')
    tot_data_L1.to_excel(writer, sheet_name='Layer1')
    tot_data_L2.to_excel(writer, sheet_name='Layer2')
    tot_data_defi.to_excel(writer, sheet_name='Defi')
    tot_data_gaming.to_excel(writer, sheet_name='Gaming')

## Trying to Export Crypto Data Using ccxt package